In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'id': [1000, 2000, 3000, 4000, 5000],
                   'name': ["Marufuku", "Jack in the Box", "Thai Basil", "Tako Sushi", "McDonald's"],
                   'cuisine': ["Japanese", "Fast Food", "Thai", "Japanese", "Fast Food"],
                   'orders': [2241, 1592, 820, 1739, 1039],
                   'rating': [4.8, 4.1, 3.9, 4.2, 4.5]})

In [3]:
df

,id,name,cuisine,orders,rating
0,1000,Marufuku,Japanese,2241,4.8
1,2000,Jack in the Box,Fast Food,1592,4.1
2,3000,Thai Basil,Thai,820,3.9
3,4000,Tako Sushi,Japanese,1739,4.2
4,5000,McDonald's,Fast Food,1039,4.5


## Anatomy of a DataFrame

In [4]:
df.index  # default index, does not always have to be this

RangeIndex(start=0, stop=5, step=1)

In [5]:
df.columns

Index(['id', 'name', 'cuisine', 'orders', 'rating'], dtype='object')

In [6]:
df.shape

(5, 5)

## Referring to Certain Parts of a DataFrame

### Accessing particular rows: `.loc` vs. `.iloc`

In [7]:
df2 = df.set_index('id')

In [8]:
df2

,name,cuisine,orders,rating
id,,,,
1000,Marufuku,Japanese,2241,4.8
2000,Jack in the Box,Fast Food,1592,4.1
3000,Thai Basil,Thai,820,3.9
4000,Tako Sushi,Japanese,1739,4.2
5000,McDonald's,Fast Food,1039,4.5


In [9]:
df

,id,name,cuisine,orders,rating
0,1000,Marufuku,Japanese,2241,4.8
1,2000,Jack in the Box,Fast Food,1592,4.1
2,3000,Thai Basil,Thai,820,3.9
3,4000,Tako Sushi,Japanese,1739,4.2
4,5000,McDonald's,Fast Food,1039,4.5


In [10]:
df.loc[1000]  # errors because 1000 is not in the index

KeyError: 1000

In [11]:
df2.loc[1000]

name       Marufuku
cuisine    Japanese
orders         2241
rating          4.8
Name: 1000, dtype: object

In [12]:
type(df2.loc[1000])  # we'll come back to what a Series is later

pandas.core.series.Series

In [13]:
df.iloc[3]

id               4000
name       Tako Sushi
cuisine      Japanese
orders           1739
rating            4.2
Name: 3, dtype: object

In [14]:
df2.iloc[3]  # this is the same as the above because both dataframes have the same restaurant in the row at index 3

name       Tako Sushi
cuisine      Japanese
orders           1739
rating            4.2
Name: 4000, dtype: object

### Accessing particular columns: `pd.Series`

In [15]:
s = df['name']
s

0           Marufuku
1    Jack in the Box
2         Thai Basil
3         Tako Sushi
4         McDonald's
Name: name, dtype: object

In [16]:
type(s)  # each column is a Series

pandas.core.series.Series

In [17]:
s.index  # a Series has an index

RangeIndex(start=0, stop=5, step=1)

In [18]:
s.shape  # a Series also has a shape

(5,)

In [19]:
s.iloc[2:4]

2    Thai Basil
3    Tako Sushi
Name: name, dtype: object

In [20]:
s.loc[2:4]

2    Thai Basil
3    Tako Sushi
4    McDonald's
Name: name, dtype: object

In [21]:
s.columns  # a Series is NOT a DataFrame!

AttributeError: 'Series' object has no attribute 'columns'

### Accessing particular rows: boolean array indexing

In [22]:
df

,id,name,cuisine,orders,rating
0,1000,Marufuku,Japanese,2241,4.8
1,2000,Jack in the Box,Fast Food,1592,4.1
2,3000,Thai Basil,Thai,820,3.9
3,4000,Tako Sushi,Japanese,1739,4.2
4,5000,McDonald's,Fast Food,1039,4.5


In [23]:
df['cuisine'] == "Japanese"  # returns what is called a "boolean array"

0     True
1    False
2    False
3     True
4    False
Name: cuisine, dtype: bool

In [24]:
df[ df['cuisine'] == "Japanese" ]

,id,name,cuisine,orders,rating
0,1000,Marufuku,Japanese,2241,4.8
3,4000,Tako Sushi,Japanese,1739,4.2


In [25]:
df.loc[ "cuisine" == "Japanese" ]  # common mistake

KeyError: False

In [26]:
df[ (df['cuisine'] == "Japanese") & (df['rating'] >= 4.5) ]  # can use more than 1 condition

,id,name,cuisine,orders,rating
0,1000,Marufuku,Japanese,2241,4.8


In [27]:
df[ df['cuisine'] == "Japanese" & df['rating'] >= 4.5 ]  # parantheses matter!

TypeError: Cannot perform 'rand_' with a dtyped [float64] array and scalar of type [bool]

## Group By Operations in Pandas

In [28]:
df

,id,name,cuisine,orders,rating
0,1000,Marufuku,Japanese,2241,4.8
1,2000,Jack in the Box,Fast Food,1592,4.1
2,3000,Thai Basil,Thai,820,3.9
3,4000,Tako Sushi,Japanese,1739,4.2
4,5000,McDonald's,Fast Food,1039,4.5


In [29]:
df.groupby('cuisine')  # this creates an intermediate object which has the groups within it

In [30]:
df.groupby('cuisine').agg(np.mean)  # notice that np.mean is used on ALL numerical columns 

,id,orders,rating
cuisine,,,
Fast Food,3500.0,1315.5,4.3
Japanese,2500.0,1990.0,4.5
Thai,3000.0,820.0,3.9


In [31]:
df.groupby('cuisine')['rating'].agg(np.mean)  # np.mean is only used on the rating column

cuisine
Fast Food    4.3
Japanese     4.5
Thai         3.9
Name: rating, dtype: float64

In [32]:
df.groupby('cuisine')[['rating']].agg(np.mean)  # if you want it as a df instead

,rating
cuisine,
Fast Food,4.3
Japanese,4.5
Thai,3.9


**YOUR TURN: Find the lowest rating within each cuisine.**

In [33]:
df.groupby('cuisine')[['rating']].agg(np.min)

,rating
cuisine,
Fast Food,4.1
Japanese,4.2
Thai,3.9


**Find the restaurant with the lowest rating within each cuisine**

In [34]:
# Implementation 1: Lambda Function
df.sort_values("rating").groupby('cuisine')['name'].agg(lambda group: group.iloc[0])

cuisine
Fast Food    Jack in the Box
Japanese          Tako Sushi
Thai              Thai Basil
Name: name, dtype: object

In [35]:
# Implementation 2: .first() aggregator
df.sort_values("rating").groupby('cuisine')['name'].first()

cuisine
Fast Food    Jack in the Box
Japanese          Tako Sushi
Thai              Thai Basil
Name: name, dtype: object

## Joins

In [36]:
left = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
   'shirt_color':['blue','blue','red','blue','green'], 
   'subject_id':['sub1','sub2','sub4','sub6','sub5']})
right = pd.DataFrame(
   {'id':[1,2,3,4,5],
   'Name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
   'shirt_color':['red','blue','red','red','blue'],
   'subject_id':['sub2','sub4','sub3','sub6','sub5']})

In [37]:
left

,id,Name,shirt_color,subject_id
0,1,Alex,blue,sub1
1,2,Amy,blue,sub2
2,3,Allen,red,sub4
3,4,Alice,blue,sub6
4,5,Ayoung,green,sub5


In [38]:
right

,id,Name,shirt_color,subject_id
0,1,Billy,red,sub2
1,2,Brian,blue,sub4
2,3,Bran,red,sub3
3,4,Bryce,red,sub6
4,5,Betty,blue,sub5


In [39]:
pd.merge(left, right, on='id')

,id,Name_x,shirt_color_x,subject_id_x,Name_y,shirt_color_y,subject_id_y
0,1,Alex,blue,sub1,Billy,red,sub2
1,2,Amy,blue,sub2,Brian,blue,sub4
2,3,Allen,red,sub4,Bran,red,sub3
3,4,Alice,blue,sub6,Bryce,red,sub6
4,5,Ayoung,green,sub5,Betty,blue,sub5


In [40]:
pd.merge(left, right, on=['id', 'shirt_color'])

,id,Name_x,shirt_color,subject_id_x,Name_y,subject_id_y
0,2,Amy,blue,sub2,Brian,sub4
1,3,Allen,red,sub4,Bran,sub3


In [41]:
pd.merge(left, right, on='shirt_color')

,id_x,Name_x,shirt_color,subject_id_x,id_y,Name_y,subject_id_y
0,1,Alex,blue,sub1,2,Brian,sub4
1,1,Alex,blue,sub1,5,Betty,sub5
2,2,Amy,blue,sub2,2,Brian,sub4
3,2,Amy,blue,sub2,5,Betty,sub5
4,4,Alice,blue,sub6,2,Brian,sub4
5,4,Alice,blue,sub6,5,Betty,sub5
6,3,Allen,red,sub4,1,Billy,sub2
7,3,Allen,red,sub4,3,Bran,sub3
8,3,Allen,red,sub4,4,Bryce,sub6


In [42]:
pd.merge(left, right, on='shirt_color', how='inner')

,id_x,Name_x,shirt_color,subject_id_x,id_y,Name_y,subject_id_y
0,1,Alex,blue,sub1,2,Brian,sub4
1,1,Alex,blue,sub1,5,Betty,sub5
2,2,Amy,blue,sub2,2,Brian,sub4
3,2,Amy,blue,sub2,5,Betty,sub5
4,4,Alice,blue,sub6,2,Brian,sub4
5,4,Alice,blue,sub6,5,Betty,sub5
6,3,Allen,red,sub4,1,Billy,sub2
7,3,Allen,red,sub4,3,Bran,sub3
8,3,Allen,red,sub4,4,Bryce,sub6


In [43]:
pd.merge(left, right, on='subject_id', how='left')

,id_x,Name_x,shirt_color_x,subject_id,id_y,Name_y,shirt_color_y
0,1,Alex,blue,sub1,NaN,NaN,NaN
1,2,Amy,blue,sub2,1.0,Billy,red
2,3,Allen,red,sub4,2.0,Brian,blue
3,4,Alice,blue,sub6,4.0,Bryce,red
4,5,Ayoung,green,sub5,5.0,Betty,blue


In [44]:
pd.merge(left, right, on='shirt_color', how='left')

,id_x,Name_x,shirt_color,subject_id_x,id_y,Name_y,subject_id_y
0,1,Alex,blue,sub1,2.0,Brian,sub4
1,1,Alex,blue,sub1,5.0,Betty,sub5
2,2,Amy,blue,sub2,2.0,Brian,sub4
3,2,Amy,blue,sub2,5.0,Betty,sub5
4,3,Allen,red,sub4,1.0,Billy,sub2
5,3,Allen,red,sub4,3.0,Bran,sub3
6,3,Allen,red,sub4,4.0,Bryce,sub6
7,4,Alice,blue,sub6,2.0,Brian,sub4
8,4,Alice,blue,sub6,5.0,Betty,sub5
9,5,Ayoung,green,sub5,NaN,NaN,NaN


In [45]:
pd.merge(left, right, on='shirt_color', how='right')

,id_x,Name_x,shirt_color,subject_id_x,id_y,Name_y,subject_id_y
0,3,Allen,red,sub4,1,Billy,sub2
1,3,Allen,red,sub4,3,Bran,sub3
2,3,Allen,red,sub4,4,Bryce,sub6
3,1,Alex,blue,sub1,2,Brian,sub4
4,2,Amy,blue,sub2,2,Brian,sub4
5,4,Alice,blue,sub6,2,Brian,sub4
6,1,Alex,blue,sub1,5,Betty,sub5
7,2,Amy,blue,sub2,5,Betty,sub5
8,4,Alice,blue,sub6,5,Betty,sub5


In [46]:
pd.merge(left, right, on='Name', how='outer')

,id_x,Name,shirt_color_x,subject_id_x,id_y,shirt_color_y,subject_id_y
0,1.0,Alex,blue,sub1,NaN,NaN,NaN
1,2.0,Amy,blue,sub2,NaN,NaN,NaN
2,3.0,Allen,red,sub4,NaN,NaN,NaN
3,4.0,Alice,blue,sub6,NaN,NaN,NaN
4,5.0,Ayoung,green,sub5,NaN,NaN,NaN
5,NaN,Billy,NaN,NaN,1.0,red,sub2
6,NaN,Brian,NaN,NaN,2.0,blue,sub4
7,NaN,Bran,NaN,NaN,3.0,red,sub3
8,NaN,Bryce,NaN,NaN,4.0,red,sub6
9,NaN,Betty,NaN,NaN,5.0,blue,sub5


## Pivot Tables

Any `pivot_table` operation has a corresponding `groupby` operation that gives the same output, but the `pivot_table` displays the output in a slightly different format. Specifically, `pivot_table` places one (or more) of the keys you group by as columns in the output DataFrame, whereas `groupby` places all of the keys you group by in the index of the output DataFrame. In some cases, `pivot_table` and `groupby` can give the exact same output.

In [47]:
data = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
data

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


In [48]:
pd.pivot_table(data, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum, fill_value=0)

C        large  small
A   B                
bar one      4      5
    two      7      6
foo one      4      1
    two      0      6

In [49]:
data.groupby(['A', 'B', 'C'])[['D']].agg(np.sum)

D
A   B   C       
bar one large  4
        small  5
    two large  7
        small  6
foo one large  4
        small  1
    two small  6

In [50]:
pd.pivot_table(data, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': np.mean})

D         E
A   C                        
bar large  5.500000  7.500000
    small  5.500000  8.500000
foo large  2.000000  4.500000
    small  2.333333  4.333333

In [51]:
data.groupby(["A", "C"])[["D", "E"]].agg(np.mean)

D         E
A   C                        
bar large  5.500000  7.500000
    small  5.500000  8.500000
foo large  2.000000  4.500000
    small  2.333333  4.333333

In [52]:
pd.pivot_table(data, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': [min, max, np.mean]})

D    E               
               mean  max      mean  min
A   C                                  
bar large  5.500000  9.0  7.500000  6.0
    small  5.500000  9.0  8.500000  8.0
foo large  2.000000  5.0  4.500000  4.0
    small  2.333333  6.0  4.333333  2.0

In [53]:
data.groupby(["A", "C"])[["D", "E"]].agg({'D': np.mean,
                                          'E': [min, max, np.mean]})

D   E              
               mean min max      mean
A   C                                
bar large  5.500000   6   9  7.500000
    small  5.500000   8   9  8.500000
foo large  2.000000   4   5  4.500000
    small  2.333333   2   6  4.333333